In [1]:
import os
os.chdir('../')

In [2]:
import re
import sys
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import transformers
import torch.nn as nn
from transformers import AdamW
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
device = torch.device("cpu")

In [4]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [5]:
dataset = pd.read_pickle('pickle/dataset.pickle')
print(dataset.shape)

(123235361, 2)


In [6]:
sentences = dataset['Sentences'].values.tolist()
tags = dataset['Tags'].values.tolist()

In [7]:
# cleaned_tags = [re.sub(r'''[',"\[\]]''', "", s) for s in tags]

# with open('cleaned_tags.pkl', 'wb') as f:
#     pickle.dump(cleaned_tags, f)

with open('cleaned_tags.pkl', 'rb') as f:
    cleaned_tags = pickle.load(f)

print(len(sentences), len(cleaned_tags))

123235361 123235361


In [8]:
unique_tags = list(set(cleaned_tags))

tag2idx = {}
for idx, tag in enumerate(unique_tags):
    tag2idx[tag] = idx

In [9]:
sentences_list = []
token_tag_list = []
sentence = []
token_tag = []
for token, tag in tqdm(zip(sentences, cleaned_tags)):
    sentence.append(token)
    token_tag.append(tag)
    try:
        if bool(re.match(r"[.]", token)):
            if len(sentence) >= 4:
                sentences_list.append(sentence)
                token_tag_list.append(token_tag)
                sentence = []
                token_tag = []
    except:
        print(f"Error in re for token: {token}")
        sentences_list.append(sentence)
        token_tag_list.append(token_tag)
        sentence = []
        token_tag = []

1831750it [00:07, 382015.51it/s]

Error in re for token: nan


2463479it [00:08, 742251.19it/s]

Error in re for token: nan
Error in re for token: nan


3333244it [00:10, 690676.49it/s]

...

Error in re for token: nan
Error in re for token: nan
Error in re for token: nan


122655365it [07:03, 705617.15it/s]

Error in re for token: nan


123235361it [07:03, 290671.57it/s]

Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
Error in re for token: nan
E

In [10]:
sentences_length = []
for val in tqdm(sentences_list):
    sentences_length.append(len(val))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5911818/5911818 [00:02<00:00, 2593087.87it/s]


In [ ]:
tokenized_sentences_list = []
tokenized_sent_tag_list = []
for token_list, tag_list in tqdm(zip(sentences_list, token_tag_list)):
    updated_token_list = []
    updated_tag_list = []
    for token, tag in zip(token_list, tag_list):
        try:
            tokenized_list = tokenizer.tokenize(token)
            updated_token_list.extend(tokenized_list)
            updated_tag_list.extend([tag2idx[tag]]*len(tokenized_list))
        except:
            print(f"Tokenizatin failed for token: {token}")
    tokenized_sentences_list.append(updated_token_list)
    tokenized_sent_tag_list.append(updated_tag_list)

81788it [01:30, 919.94it/s] 

Tokenizatin failed for token: nan


111672it [02:05, 926.44it/s] 

Tokenizatin failed for token: nan


113539it [02:07, 928.52it/s]

...

5157027it [1:36:47, 972.96it/s]

Tokenizatin failed for token: nan


5186009it [1:37:16, 994.57it/s] 

Tokenizatin failed for token: nan


5225877it [1:37:56, 992.52it/s] 

In [ ]:
# Storing Tokenized Data
with open('pickle/tokenized_sentences_list.pkl', 'wb') as f:
    pickle.dump(tokenized_sentences_list, f)
with open('pickle/tokenized_sent_tag_list.pkl', 'wb') as f:
    pickle.dump(tokenized_sent_tag_list, f)
    
# # Loading Processed Tokenized Data
# with open('pickle/tokenized_sentences_list.pkl', 'rb') as f:
#     tokenized_sentences_list = pickle.load(f)
    
# with open('pickle/tokenized_sent_tag_list.pkl', 'rb') as f:
#     tokenized_sent_tag_list = pickle.load(f)

In [ ]:
# Mapping tokens to ids
input_ids = []
for tokenized_sentence in tqdm(tokenized_sentences_list):
    input_ids.append(tokenizer.convert_tokens_to_ids(tokenized_sentence))

In [ ]:
len(input_ids), len(tokenized_sent_tag_list)

In [ ]:
attention_mask = []
for input_ in tqdm(input_ids):
    attention_mask.append(torch.ones(len(input_)))

padded_attention_mask = torch.nn.utils.rnn.pad_sequence(
    attention_mask, 
    batch_first=True, 
    padding_value=0.0)

padded_input_ids = torch.nn.utils.rnn.pad_sequence(
    [torch.tensor(input_) for input_ in input_ids], 
    batch_first=True, 
    padding_value=0.0)

padded_tags = torch.nn.utils.rnn.pad_sequence(
    [torch.tensor(tag_) for tag_ in tokenized_sent_tag_list], 
    batch_first=True, 
    padding_value=0.0)

In [ ]:
class NERBert(nn.Module):
    
    def __init__(self, tag_count=4):
        super(NERBert, self).__init__()
        self.bert = AutoModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, tag_count)
    
    def forward(self, input_ids, attention_mask):
        
        output = self.bert(input_ids, attention_mask=attention_mask) # Model gives 'last_hidden_state' and 'pooler_output'
        
        pre_classifier_layer = self.dropout(output.last_hidden_state)
        model_output = self.classifier(pre_classifier_layer)
        
        return model_output

In [ ]:
train_tokens, temp_tokens, train_tags, temp_tags, train_mask, temp_mask =  train_test_split(
    padded_input_ids, padded_tags, padded_attention_mask,
    random_state=2018, 
    test_size=0.3
)

val_tokens, test_tokens, val_tags, test_tags, val_mask, test_mask = train_test_split(
    temp_tokens, temp_tags, temp_mask,
    random_state=2018, 
    test_size=0.5
)

In [ ]:
batch_size = 32
# wrap tensors
train_data = TensorDataset(train_tokens, train_mask, train_tags)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_tokens, val_mask, val_tags)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:
def train(model, optimizer, loss_criteria, train_dataloader):
    try:
        model.train()
        
        total_loss = 0
        total_logits = []
        
        # iterate over batches
        for step,batch in enumerate(train_dataloader):

            # progress update after every 50 batches.
            if step % 50 == 0 and not step == 0:
                print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

            # push the batch to gpu
            batch = [r.to(device) for r in batch]

            sent_id, mask, labels = batch

            # clear previously calculated gradients 
            model.zero_grad()        

            # get model predictions for the current batch
            logits = model(sent_id, mask)

            # compute the loss between actual and predicted values
            loss = loss_criteria(logits.permute(0, 2, 1), labels)

            # add on to the total loss
            total_loss = total_loss + loss.item()

            # backward pass to calculate the gradients
            loss.backward()

            # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # update parameters
            optimizer.step()

            # model predictions are stored on GPU. So, push it to CPU
            logits = logits.detach().cpu().numpy()

            # append the model predictions
            total_logits.append(logits)
            
        # compute the training loss of the epoch
        avg_loss = total_loss / len(train_dataloader)
        
        total_logits = np.concatenate(total_logits, axis=0)
        

        return avg_loss, total_logits
    except Exception as e:
        print(f"Error during training the model on line: {sys.exc_info()[2].tb_lineno}")
        print(e)

In [ ]:
# function for evaluating the model
def evaluate(model, val_dataloader, loss_criteria):
  
    print("\nEvaluating...")

    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_logits = []

    # iterate over batches
    for step,batch in enumerate(val_dataloader):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:

            # Calculate elapsed time in minutes.
#             elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            logits = model(sent_id, mask)

            # compute the validation loss between actual and predicted values
            loss = loss_criteria(logits.permute(0, 2, 1),labels)

            total_loss = total_loss + loss.item()

            logits = logits.detach().cpu().numpy()

            total_logits.append(logits)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

    # reshape the predictions in form of (number of samples, no. of classes)
    total_logits  = np.concatenate(total_logits, axis=0)

    return avg_loss, total_logits

In [ ]:
model = NERBert(tag_count=len(tag2idx))
model = model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5) # learning rate

criterion = nn.CrossEntropyLoss()

In [ ]:
epochs=50
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train(
        model = model, 
        optimizer = optimizer, 
        loss_criteria = criterion, 
        train_dataloader = train_dataloader 
        
    )
    
    #evaluate model
    valid_loss, _ = evaluate(
        model = model, 
        val_dataloader = val_dataloader, 
        loss_criteria = criterion
    )
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'trained_weights/saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

In [ ]:
def get_prediction_from_logits(logits):
    try:
        tag_prob = nn.Softmax(dim=2)(logits)
        tag_prediction = torch.argmax(tag_prob, dim=2).detach().cpu().numpy()
        return tag_prediction
    except Exception as e:
        print(f"Error in line: {sys.exc_info()[2].tb_lineno}")
        print(e)

In [ ]:
def classification_result(tag2idx, c_tag_id):
    try:
        prediction_result = []
        for sent_ in c_tag_id:
            prediction_result.append(
                list(map(lambda x: list(tag2idx.keys())[list(tag2idx.values()).index(x)], sent_))
            )
            
        tagged_entity = np.concatenate(prediction_result, axis=0)
        return tagged_entity
    except Exception as e:
        print(f"Error in line: {sys.exc_info()[2].tb_lineno}")
        print(e)        

In [ ]:
#load weights of best model
path = 'trained_weights/saved_weights.pt'
model.load_state_dict(torch.load(path))

In [ ]:
# get predictions for test data
with torch.no_grad():
    logits = model(test_tokens.to(device), test_mask.to(device))
    preds = get_prediction_from_logits(logits=logits)

In [ ]:
test_tags = test_tags.detach().cpu().numpy()

In [ ]:
test_tags = classification_result(
    tag2idx = tag2idx, 
    c_tag_id = test_tags
)
preds = classification_result(
    tag2idx = tag2idx, 
    c_tag_id = preds
)

In [ ]:
test_tags.shape, preds.shape

In [ ]:
print(classification_report(np.squeeze(test_tags.reshape(1, -1)), np.squeeze(preds.reshape(1, -1))))